In [2]:
import numpy as np
from timeit import default_timer as timer
from matplotlib import pyplot
import math
import numbapro # We use check_cuda() and vectorize
from numbapro import vectorize, autojit, jit, cuda #vectorize for functions and cuda for cuda-u functions
numbapro.check_cuda()

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
numbapro:1: ImportWarning: The numbapro package is deprecated in favour of the accelerate package. Please update your code to use equivalent functions from accelerate.


------------------------------libraries detection-------------------------------
Finding cublas
	located at /home/john/anaconda2/lib/libcublas.so.7.0.28
	trying to open library...	ok
Finding cusparse
	located at /home/john/anaconda2/lib/libcusparse.so.7.0.28
	trying to open library...	ok
Finding cufft
	located at /home/john/anaconda2/lib/libcufft.so.7.0.35
	trying to open library...	ok
Finding curand
	located at /home/john/anaconda2/lib/libcurand.so.7.0.28
	trying to open library...	ok
Finding nvvm
	located at /home/john/anaconda2/lib/libnvvm.so.3.0.0
	trying to open library...	ok
	finding libdevice for compute_20...	ok
	finding libdevice for compute_30...	ok
	finding libdevice for compute_35...	ok
-------------------------------hardware detection-------------------------------
Found 1 CUDA devices
id 0        GeForce GTX 870M                              [SUPPORTED]
                      compute capability: 3.0
                           pci device id: 0
                              

True

In [3]:
import numba.cuda  
#this line from nbviewer.ipython.org/github/ContinuumIO/numbapro-examples/blob/master/webinars/2014_06_17/intro_to_gpu_python.ipynb
my_gpu = numba.cuda.get_current_device()
print "Running on GPU:", my_gpu.name
cores_per_capability = {
    1: 8,
    2: 32,
    3: 192,
}
cc = my_gpu.compute_capability
print "Compute capability: ", "%d.%d" % cc, "(Numba requires >= 2.0)"
majorcc = cc[0]
print "Number of streaming multiprocessor:", my_gpu.MULTIPROCESSOR_COUNT
cores_per_multiprocessor = cores_per_capability[majorcc]
print "Number of cores per mutliprocessor:", cores_per_multiprocessor
total_cores = cores_per_multiprocessor * my_gpu.MULTIPROCESSOR_COUNT
print "Number of cores on GPU:", total_cores

Running on GPU: GeForce GTX 870M
Compute capability:  3.0 (Numba requires >= 2.0)
Number of streaming multiprocessor: 7
Number of cores per mutliprocessor: 192
Number of cores on GPU: 1344


First, let's do an example, in serial and then GPU accelerate it. 

In [3]:
def VectorAdd(a, b, c):
    for i in xrange(a.size):
        c[i] = a[i] + b[i]

In [4]:
def main():
    N = 32000000
    
    A = np.ones(N, dtype = np.float32)
    B = np.ones(N, dtype = np.float32)
    C = np.zeros(N, dtype = np.float32)
    
    start = timer()
    VectorAdd(A, B, C)
    vectoradd_time = timer() - start
    
    print("C[:5] = " + str(C[:5]))
    print("C[:-5] = " + str(C[:-5]))
    
    print("VectorAdd took %f seconds" % vectoradd_time)
    
if __name__ == '__main__':
    main()

C[:5] = [ 2.  2.  2.  2.  2.]
C[:-5] = [ 2.  2.  2. ...,  2.  2.  2.]
VectorAdd took 7.216313 seconds


Now let's GPUize the computation with @vectorize:

In [5]:
#simple scalar addition, matrix multiplication can be performed with guvectorize
@vectorize(["float32(float32, float32)"], target='gpu') 
def GPU_VectorAdd(a, b):
    return a + b  

In [6]:
def main():
    N = 32000000
    
    A = np.ones(N, dtype = np.float32)
    B = np.ones(N, dtype = np.float32)
    C = np.zeros(N, dtype = np.float32)
    
    start = timer()
    C = GPU_VectorAdd(A, B) #this line is different
    vectoradd_time = timer() - start
    
    print("C[:5] = " + str(C[:5]))
    print("C[:-5] = " + str(C[:-5]))
    
    print("GPU_VectorAdd took %f seconds" % vectoradd_time)
    
if __name__ == '__main__':
    main()

C[:5] = [ 2.  2.  2.  2.  2.]
C[:-5] = [ 2.  2.  2. ...,  2.  2.  2.]
GPU_VectorAdd took 0.073357 seconds


Now let's write a function that calculates the correlation function, but we should monitor the GPU memory usage

In [5]:
def iter_loadtxt(filename, delimiter=',', skiprows=1, dtype=float):
    def iter_func():
        with open(filename, 'r') as infile:
            for _ in range(skiprows):
                next(infile)
            for line in infile:
                line = line.rstrip().split(delimiter)
                for item in line:
                    yield dtype(item)
        iter_loadtxt.rowlength = len(line)

    data = np.fromiter(iter_func(), dtype=dtype)
    data = data.reshape((-1, iter_loadtxt.rowlength))
    return data

In [6]:
#This code will import nodal polar vector values
file1 = iter_loadtxt('/media/john/My Passport/slabs/serge_tests/data0.csv',delimiter=',',skiprows=1,dtype=float)

In [7]:
polar0 = [row[0] for row in file1]
polar1 = [row[1] for row in file1]
polar2 = [row[2] for row in file1]

r0 = [row[5] for row in file1]
r1 = [row[6] for row in file1]
r2 = [row[7] for row in file1]

In [118]:
import time
polar0ave = np.mean(polar0);
polar1ave = np.mean(polar1);
polar2ave = np.mean(polar2);

N = 150;

Cr = np.zeros(len(polar0));
R = np.zeros(len(polar0));

#use the correlation function of 10.1103/PhysRevLett.91.267204 (not volume normalized)
#this calculation is O(N(N-1))

start = time.time()
for i in range(1,len(polar0)):
    for j in range(1,N):
        #if i != j:
        Cr[i] += 1/(0.8*0.8)*((polar0[i] - polar0ave)* (polar0[j] - polar0ave) + (polar1[i] - polar1ave) * (polar1[j] - polar1ave) + (polar2[i] - polar2ave) * (polar2[j] - polar2ave));
    R[i] = (r0[i]*r0[i] +  r1[i]*r1[i] + r2[i]*r2[i])**(0.5)
    
np.savetxt("/media/john/My Passport/slabs/serial_Cr.csv", Cr, delimiter=",")
np.savetxt("/media/john/My Passport/slabs/serial_R.csv", R, delimiter=",")    
end = time.time()
print end - start

27.1577670574


In [15]:
@autojit
def myAdd(a, b):
    return a + b

In [32]:
@vectorize(["float64(float64, float64, float64, float64, float64, float64)"], target='gpu') 
def Corrfunct(polar0f, polar1f, polar2f, f0, f1, f2):
    return 1/(0.8*0.8)*(polar0f * f0 + polar1f * f1 + polar2f * f2);

In [123]:
def main():
    #N = len(polar0)-1
    N = 3000
    Polar0fluct = np.asarray([row[0] for row in file1] - np.mean(polar0))
    Polar1fluct = np.asarray([row[1] for row in file1] - np.mean(polar1))
    Polar2fluct = np.asarray([row[2] for row in file1] - np.mean(polar2))

    R0 = np.asarray([row[5] for row in file1])
    R1 = np.asarray([row[6] for row in file1])
    R2 = np.asarray([row[7] for row in file1])

    R = np.zeros(len(polar0), dtype = np.float64);
    Cr = np.zeros((len(polar0)), dtype = np.float64);

    
    start = timer()
    R = Rfunct(R0, R1, R2) 
    
    # below we take use the GPU at every stage in the loop and only require one processor 
    # to mediate the transfer of the memory. According to CUDA documentation, NumPy 
    # automatically transfers memory when the GPU calls for it. There is still this strange
    # issue of needing to correlate N by (N-1) 
    
    for k in xrange(N):
        Cr += Corrfunct(Polar0fluct, Polar1fluct, Polar2fluct, Polar0fluct[k], Polar1fluct[k], Polar2fluct[k])
    
    np.savetxt("/media/john/My Passport/slabs/GPU_Cr.csv", Cr, delimiter=",")
    np.savetxt("/media/john/My Passport/slabs/GPU_R.csv", R, delimiter=",")
    vectoradd_time = timer() - start
    
    print("R[:5] = " + str(R[:5]))
    print("R[:-5] = " + str(R[:-5]))
    
    print("Cr[:1] = " + str(Cr[:1]))
    print("Cr[:2] = " + str(Cr[:2]))
    
    print("Correlation function took %f seconds" % vectoradd_time)
    
if __name__ == '__main__':
    main()

R[:5] = [ 57.82732918  57.14017851  56.44466317  57.14017851  57.67968805]
R[:-5] = [ 57.82732918  57.14017851  56.44466317 ...,  53.22593353  53.85164807
  54.4885309 ]
Cr[:1] = [ 184.75990523]
Cr[:2] = [ 184.75990523   -3.20883298]
Correlation function took 10.840551 seconds


In [107]:
#something is wrong here... why are we at 0? It isn't saving the result!

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [100]:
Cr

[]

In [130]:
myAdd_gpu = cuda.jit(restype=f8, argtypes=[f8, f8], device=True)(myAdd)

NameError: name 'f8' is not defined

In [134]:
a = range(0,5)

In [97]:

type(Cr)

list

In [144]:
import pycuda

ImportError: No module named pycuda

In [102]:
N = 50
Polar0fluct = np.asarray([row[0] for row in file1] - np.mean(polar0))
Polar1fluct = np.asarray([row[1] for row in file1] - np.mean(polar1))
Polar2fluct = np.asarray([row[2] for row in file1] - np.mean(polar2))

R0 = np.asarray([row[5] for row in file1])
R1 = np.asarray([row[6] for row in file1])
R2 = np.asarray([row[7] for row in file1])

R = np.zeros(len(polar0), dtype = np.float64);
Cr = np.zeros((len(polar0)), dtype = np.float64);

    
start = timer()
R = Rfunct(R0, R1, R2) #this line is different
for k in xrange(N):
    Cr += Corrfunct(Polar0fluct, Polar1fluct, Polar2fluct, Polar0fluct[k], Polar1fluct[k], Polar2fluct[k])
vectoradd_time = timer() - start
    
print("R[:5] = " + str(R[:5]))
print("R[:-5] = " + str(R[:-5]))
    
print("Cr[:1] = " + str(Cr[:1]))
print("Cr[:2] = " + str(Cr[:2]))
    
print("Correlation function took %f seconds" % vectoradd_time)
    
np.savetxt("/media/john/My Passport/slabs/GPU_Cr.csv", Cr, delimiter=",")
np.savetxt("/media/john/My Passport/slabs/GPU_R.csv", R, delimiter=",")
    

R[:5] = [ 57.82732918  57.14017851  56.44466317  57.14017851  57.67968805]
R[:-5] = [ 57.82732918  57.14017851  56.44466317 ...,  53.22593353  53.85164807
  54.4885309 ]
Cr[:1] = [-1.65168475]
Cr[:2] = [-1.65168475  4.37388181]
Correlation function took 0.176346 seconds


In [104]:
len(Cr)

170586